In [1]:
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from time import sleep as para
from time import time as contar
import datetime as dt

import pandas as pd

from openpyxl import load_workbook, Workbook
from tkinter import messagebox
from tkinter import Tk

In [2]:
class Navegacao():
    def __init__(self):

        servico = Service(ChromeDriverManager().install())

        ask = messagebox.askyesno("Esconder Navegador", "Deseja esconder o navegador?")
        janela = Tk()
        if ask:
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")
            navegador = webdriver.Chrome(options=options, service=servico)
        else: navegador = webdriver.Chrome(service=servico)
        janela.destroy()
        self.vars = {'ignored_exceptions': (NoSuchElementException,StaleElementReferenceException), 
                    'senha': f'Jaspekde27.{dt.date.today().month*3}'}
        self.navegador = navegador
        

In [3]:
driver = Navegacao()
driver.navegador.get('https://www.google.com/')
driver.navegador.close()

In [ ]:
import os

def ler_arquivos_em_pasta(caminho_pasta):
    arquivos = os.listdir(caminho_pasta)
    
    for arquivo in arquivos:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        
        # if '.doc' in os.path.isfile(caminho_arquivo):
        if '.doc' in arquivo and os.path.isfile(caminho_arquivo):
            # with open(caminho_arquivo, 'r') as f:
            #     conteudo = f.read()
            #     # Faça algo com o conteúdo do arquivo
            #     print(conteudo)
            print(arquivo, type(arquivo))
            novo_nome = os.path.splitext(arquivo)[0] + ".docx"
            print(novo_nome, type(novo_nome))
            os.rename(caminho_arquivo, os.path.join(caminho_pasta, novo_nome))

# Exemplo de uso
caminho_pasta = r"\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS\CONTRATO_2014"
ler_arquivos_em_pasta(caminho_pasta)
# print(d)

In [ ]:
from docx import Document

def ler_arquivo_docx(caminho_arquivo):
    doc = Document(caminho_arquivo)
    
    conteudo = ""
    for paragrafo in doc.paragraphs:
        conteudo += paragrafo.text + "\n"
    
    return conteudo

# Exemplo de uso
# caminho_arquivo = doc
# caminho_arquivo = r"\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS\CONTRATO_2014\CONTRATO 001 -WT CONS-10-FEV.doc"
caminho_arquivo = "\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS\CONTRATO_2014\CONTRATO 006-LAGHI-ABRIL.docx"
texto = ler_arquivo_docx(caminho_arquivo)
print(texto)


In [12]:
import win32com.client

def ler_arquivo_doc(caminho_arquivo_doc):
    word = win32com.client.Dispatch("Word.Application")
    doc = word.Documents.Open(caminho_arquivo_doc)
    
    # Extrai o texto do documento .doc
    texto = doc.Content.Text
    
    # Fecha o documento e o aplicativo Word
    doc.Close()
    word.Quit()
    
    return texto

# Exemplo de uso
caminho_arquivo_doc = r"\\soturno\CONTABILIDADE\08 - PLANILHAS OBRAS\CONTRATOS\CONTRATO_2014\CONTRATO 001 -WT CONS-10-FEV.doc"
texto_do_doc = ler_arquivo_doc(caminho_arquivo_doc)
texto_do_doc


'\r\r\rEXTRATO\r\r\r1.ESPÉCIE: Contrato nº. 001, celebrado em10.02.2014.\r2.PARTES: Município de Manaus (SEMINF) e a empresa  WT CONSTRUÇÕES E COMÉRCIO LTDA.\r3.OBJETO: Revitalização e Adequação do Parque Municipal do Mindú em  Manaus/AM, de acordo com o Edital da Tomada de Preços nº. 005/2013-CML/PM e seus anexos.\r4.VALOR GLOBAL: R$ 878.094,10 (Oitocentos e setenta e oito mil, noventa e quatro reais e dez centavos. \r5.DOTAÇÃO: 28900 – 18.541.1081.2187.0000 - 44905120, Fonte nº. 06100000, Nota de Empenho nº. 00098 de 30.012014.\r6.PRAZO: O prazo de vigência do presente contrato será de 180 (cento e oitenta) dias corridos.\r\rManaus, 10 de fevereiro de 2014.\r\r\r\rORLANDO CABRAL HOLANDA\rSubsecretário Municipal de Obras Públicas -\rSEMINF\r\r'